# EWMAC Strategy Backtesting Example

This notebook demonstrates how to backtest the EWMAC (Exponentially Weighted Moving Average Crossover) strategy using the Quants Lab framework.

**Strategy Overview:**
- Generates long signals when fast EMA crosses above slow EMA
- Generates short signals when fast EMA crosses below slow EMA
- Exits positions on opposite crossover signals
- Uses fixed position sizing

**Author:** Quants Lab Team  
**Date:** 2025-09-29

## 1. Setup and Imports

In [5]:
import os
import sys
from datetime import datetime, timedelta
from decimal import Decimal
import pandas as pd
import numpy as np
from pathlib import Path


# Find repo root (folder that contains /src)
p = Path.cwd().resolve()
while p != p.parent and not (p / "src").exists():
    p = p.parent

# Add both ROOT (for 'hummingbot') and SRC (for 'core')
root_path = str(p)
src_path  = str(p / "src")
for path in (root_path, src_path):
    if path not in sys.path:
        sys.path.insert(0, path)

print("sys.path[0:3] =", sys.path[:3])

# Sanity imports
import hummingbot
print("hummingbot at:", getattr(hummingbot, "__file__", "(namespace package)"))

# Try connector_base
try:
    import hummingbot.connector.connector_base as cb
    print("connector_base at:", cb.__file__)
except Exception as e:
    print("connector_base import error:", repr(e))


# Add paths for imports
# sys.path.append('../')
# Walk up until we find the repo root (the one that contains /src)
p = Path.cwd().resolve()
while p != p.parent and not (p / "src").exists():
    p = p.parent

# Put ROOT (for `hummingbot/`) and SRC (for `core/`) on the path
root_path = str(p)
src_path  = str(p / "src")

for path in (root_path, src_path):
    if path not in sys.path:
        sys.path.insert(0, path)

# Import Quants Lab components
from core.backtesting import BacktestingEngine
from core.data_structures import BacktestingResult

# Import EWMAC strategy components
from strategies.Testing.ewmac_config import EWMACControllerConfig
from strategies.Testing.ewmac_controller import EWMACController

print("Imports successful!")

sys.path[0:3] = ['C:\\Users\\aljaz\\Dev\\Projects\\quant_lab', 'C:\\Users\\aljaz\\Dev\\Projects\\quant_lab\\src', 'c:\\src']
hummingbot at: None
connector_base import error: ModuleNotFoundError("No module named 'hummingbot.connector'")


ModuleNotFoundError: No module named 'hummingbot.connector'

## 2. Define Backtesting Parameters

In [ ]:
# Define backtesting period
end_date = datetime.now()
start_date = end_date - timedelta(days=30)  # 30 days of data

# Convert to timestamps
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())

print(f"Backtesting Period:")
print(f"Start: {start_date.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"End: {end_date.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Duration: {(end_date - start_date).days} days")

## 3. Configure EWMAC Strategy

In [ ]:
# Create strategy configuration
config = EWMACControllerConfig(
    # Strategy identification
    id="ewmac_eth_usdt_backtest",
    
    # Exchange and trading pair
    connector_name="binance_paper_trade",
    trading_pair="ETH-USDT",
    
    # EMA parameters
    fast_ema_period=8,
    slow_ema_period=32,
    
    # Position sizing
    position_size_quote=Decimal("100"),  # $100 per position
    total_amount_quote=Decimal("1000"),   # $1000 total capital
    
    # Risk management
    max_executors_per_side=1,
    cooldown_time=60,  # 60 seconds cooldown
    
    # Signal filtering
    min_signal_strength=0.0,  # Accept all signals
    
    # Candles configuration
    interval="1m",
)

print("Strategy Configuration:")
print(f"  Trading Pair: {config.trading_pair}")
print(f"  EMA Periods: {config.fast_ema_period}/{config.slow_ema_period}")
print(f"  Position Size: ${config.position_size_quote}")
print(f"  Total Capital: ${config.total_amount_quote}")
print(f"  Interval: {config.interval}")

## 4. Initialize Backtesting Engine

In [ ]:
# Initialize backtesting engine
engine = BacktestingEngine(
    root_path="../",  # Path to project root
    load_cached_data=True,  # Use cached candle data if available
)

print("Backtesting engine initialized!")

## 5. Run Backtest

In [ ]:
# Run backtest
print("Running backtest...")
result = await engine.run_backtesting(
    config=config,
    start=start_timestamp,
    end=end_timestamp,
    backtesting_resolution="1m",
    trade_cost=0.0006  # 0.06% trading fee
)

print("Backtest completed!")

## 6. Display Results Summary

In [ ]:
# Get results summary
summary = result.get_results_summary()

print("="*60)
print("BACKTEST RESULTS SUMMARY")
print("="*60)
print(f"\nPerformance Metrics:")
print(f"  Net PnL (Quote): ${summary['net_pnl_quote']:.2f}")
print(f"  Net PnL (%): {summary['net_pnl_pct']:.2%}")
print(f"  Max Drawdown (USD): ${summary['max_drawdown_usd']:.2f}")
print(f"  Max Drawdown (%): {summary['max_drawdown_pct']:.2%}")
print(f"  Sharpe Ratio: {summary['sharpe_ratio']:.2f}")
print(f"  Profit Factor: {summary['profit_factor']:.2f}")

print(f"\nTrading Activity:")
print(f"  Total Executors: {summary['total_executors']}")
print(f"  Total Volume: ${summary['total_volume']:.2f}")
print(f"  Long Accuracy: {summary['accuracy_long']:.2%}")
print(f"  Short Accuracy: {summary['accuracy_short']:.2%}")

print(f"\nClose Types:")
close_types = summary.get('close_types', {})
for close_type, count in close_types.items():
    print(f"  {close_type}: {count}")
print("="*60)

## 7. Visualize Results

In [ ]:
# Plot backtesting results
fig = result.get_backtesting_figure()
fig.show()

print("Chart displayed above shows:")
print("  Top panel: Price action with entry/exit points")
print("  Bottom panel: Cumulative PnL over time")

## 8. Analyze Individual Trades

In [ ]:
# Get executor data
executors_df = pd.DataFrame([e.__dict__ for e in result.executors])

if len(executors_df) > 0:
    print(f"\nTotal Trades: {len(executors_df)}")
    print("\nFirst 5 Trades:")
    print(executors_df[['side', 'entry_price', 'exit_price', 'net_pnl_quote', 'close_type']].head())
    
    # Trade statistics
    winning_trades = len(executors_df[executors_df['net_pnl_quote'] > 0])
    losing_trades = len(executors_df[executors_df['net_pnl_quote'] <= 0])
    avg_win = executors_df[executors_df['net_pnl_quote'] > 0]['net_pnl_quote'].mean()
    avg_loss = executors_df[executors_df['net_pnl_quote'] <= 0]['net_pnl_quote'].mean()
    
    print(f"\nTrade Statistics:")
    print(f"  Winning Trades: {winning_trades}")
    print(f"  Losing Trades: {losing_trades}")
    print(f"  Win Rate: {winning_trades / len(executors_df):.2%}")
    print(f"  Avg Win: ${avg_win:.2f}")
    print(f"  Avg Loss: ${avg_loss:.2f}")
    print(f"  Risk/Reward Ratio: {abs(avg_win / avg_loss):.2f}")
else:
    print("No trades executed during backtest period.")

## 9. Parameter Sensitivity Analysis (Optional)

In [ ]:
# Test different EMA period combinations
print("Running parameter sensitivity analysis...\n")

ema_combinations = [
    (5, 15),
    (8, 21),
    (8, 32),
    (12, 26),
    (20, 50),
]

results_list = []

for fast, slow in ema_combinations:
    # Update config
    test_config = config.copy()
    test_config.fast_ema_period = fast
    test_config.slow_ema_period = slow
    test_config.id = f"ewmac_{fast}_{slow}"
    
    # Run backtest
    test_result = await engine.run_backtesting(
        config=test_config,
        start=start_timestamp,
        end=end_timestamp,
        backtesting_resolution="1m",
        trade_cost=0.0006
    )
    
    # Store results
    summary = test_result.get_results_summary()
    results_list.append({
        'fast_ema': fast,
        'slow_ema': slow,
        'net_pnl_pct': summary['net_pnl_pct'],
        'sharpe_ratio': summary['sharpe_ratio'],
        'max_drawdown_pct': summary['max_drawdown_pct'],
        'total_executors': summary['total_executors'],
    })
    
    print(f"EMA {fast}/{slow}: PnL={summary['net_pnl_pct']:.2%}, Sharpe={summary['sharpe_ratio']:.2f}")

# Display results
sensitivity_df = pd.DataFrame(results_list)
print("\nParameter Sensitivity Results:")
print(sensitivity_df.to_string(index=False))

## 10. Export Results (Optional)

In [ ]:
# Export results to CSV
output_dir = "../backtesting_results/ewmac/"
os.makedirs(output_dir, exist_ok=True)

# Export executor data
if len(executors_df) > 0:
    filename = f"ewmac_trades_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    executors_df.to_csv(os.path.join(output_dir, filename), index=False)
    print(f"Trades exported to: {output_dir}{filename}")

# Export summary
summary_filename = f"ewmac_summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
import json
with open(os.path.join(output_dir, summary_filename), 'w') as f:
    json.dump(summary, f, indent=2, default=str)
print(f"Summary exported to: {output_dir}{summary_filename}")

## Next Steps

After reviewing the backtest results:

1. **Optimize Parameters**: Try different EMA periods, position sizes, or signal strength thresholds
2. **Walk-Forward Analysis**: Use `WalkForwardOptimizer` for robust parameter selection
3. **Monte Carlo Simulation**: Test strategy robustness under different market conditions
4. **Live Testing**: Deploy to paper trading for real-time validation
5. **Production**: Deploy via Hummingbot Dashboard for live trading

**Documentation:**
- Strategy details: `strategies/EWMAC/README.md`
- Optimization guide: `research_notebooks/optimization/`
- Deployment guide: `docs/deployment.md`